# Read data

In [15]:
import csv
import numpy as np
import cv2
import tensorflow as tf

from sklearn.model_selection import train_test_split
import sklearn

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D
# from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [2]:
import warnings
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. You can drastically improve your network training by using a GPU.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [4]:

lines =[]
# with open("../data-car-behavioral/driving_log.csv") as csvfile:
#     reader = csv.reader(csvfile)
#     for line in reader:
#     	lines.append(line)

# images = []
# measurements = []
# for line in lines:
# 	source_path = line[0]
# 	filename = source_path.split('/')[-1]
# 	current_path = '../data-car-behavioral/IMG/' + filename
# 	image = cv2.imread(current_path)
# 	images.append(image)
# 	# fleeping images & steering measurements
# 	measurement = float(line[3])
# 	measurements.append(measurement)

# X_train = np.array(images)
# y_train = np.array(measurements)

# images = []
# measurements = []
# for line in lines:
# 	source_path = line[0]
# 	filename = source_path.split('/')[-1]
# 	current_path = '../data-car-behavioral/IMG/' + filename
# 	image = cv2.imread(current_path)
# 	images.append(image)
# 	# fleeping images & steering measurements
# 	measurement = float(line[3])
# 	measurements.append(measurement)

# # data augmentation
# augmented_images, augmented_measurements = [], []
# for image, measurement in zip(images, measurements):
# 	augmented_images.append(image)
# 	augmented_measurements.append(measurement)

# 	augmented_images.append(cv2.flip(image, 1))
# 	augmented_measurements.append(measurement * -1.0)

# X_train = np.array(augmented_images)
# y_train = np.array(augmented_measurements)

car_images = []
steering_angles = []
with open("../data-car-behavioral/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        # lines.append(line)
        steering_center = float(line[3])

        # create adjusted steering measurements for the side camera images
        correction = steering_center * 0.1 # 10%
        steering_left = steering_center + correction
        steering_right = steering_center - correction

        # read in images from center, left and right cameras
        for i in range(3):
            source_path = line[i]
            filename = source_path.split('/')[-1]
            current_path = '../data-car-behavioral/IMG/' + filename
            image = cv2.imread(current_path)
            # add images to data set
            car_images.append(image)

        # add angles to data set	    	
        steering_angles.extend([steering_center, steering_left, steering_right])

# Data augmentation

## Flipping images

In [5]:
# data augmentation
augmented_images, augmented_measurements = [], []
for image, measurement in zip(car_images, steering_angles):
    augmented_images.append(image)
    augmented_measurements.append(measurement)

    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement * -1.0)

# Train

In [6]:
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

## LeNet architecture

In [14]:
def leNet(X_train, y_train):
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    model.add(Conv2D(6, (5, 5), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(6, (5, 5), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(84))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam')
    model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

    model.save('model_lenet.h5')
    print("Model saved as model_lenet.h5")
    

In [15]:
leNet(X_train, y_train)

Train on 13195 samples, validate on 3299 samples
Epoch 1/5
13195/13195 [==============================] - 24s 2ms/step - loss: 0.1742 - val_loss: 0.1045
Epoch 2/5
13195/13195 [==============================] - 24s 2ms/step - loss: 0.0632 - val_loss: 0.0941
Epoch 3/5
13195/13195 [==============================] - 24s 2ms/step - loss: 0.0585 - val_loss: 0.0930
Epoch 4/5
13195/13195 [==============================] - 24s 2ms/step - loss: 0.0555 - val_loss: 0.0892
Epoch 5/5
13195/13195 [==============================] - 24s 2ms/step - loss: 0.0528 - val_loss: 0.0902
Model saved as model_lenet.h5


## NVidia CNN architecture

In [13]:
img_shape = X_train[0].shape
print(img_shape)

(160, 320, 3)


Not all of these pixels contain useful information, however. In the image above, the top portion of the image captures trees and hills and sky, and the bottom portion of the image captures the hood of the car.

Your model might train faster if you crop each image to focus on only the portion of the image that is useful for predicting a steering angle.

In [20]:
def Nvidia_CNN(X_train, y_train, input_shape=(160, 320, 3)):
    model = Sequential()
    # Preprocess incoming data, centered around zero with small standard deviation 
    model.add(Lambda(lambda x: x/127.5 - 1.,
             input_shape=input_shape))
    # crop images 
#     model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    
    # adding 5 convolutions 
    model.add(Conv2D(24, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(36, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1164, kernel_initializer='he_normal'))
    model.add(Dense(100, kernel_initializer='he_normal'))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Dense(1, kernel_initializer='he_normal'))

    model.compile(loss='mse', optimizer='adam')
    model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

    model.save('model_nvidia.h5')
    print("Model saved as model_nvidia.h5")

In [21]:
Nvidia_CNN(X_train, y_train, input_shape=img_shape)

Train on 13195 samples, validate on 3299 samples
Epoch 1/5
13195/13195 [==============================] - 84s 6ms/step - loss: 44.2006 - val_loss: 0.1140
Epoch 2/5
13195/13195 [==============================] - 82s 6ms/step - loss: 0.0705 - val_loss: 0.1140
Epoch 3/5
13195/13195 [==============================] - 82s 6ms/step - loss: 0.0677 - val_loss: 0.0982
Epoch 4/5
13195/13195 [==============================] - 82s 6ms/step - loss: 0.0676 - val_loss: 0.0958
Epoch 5/5
13195/13195 [==============================] - 82s 6ms/step - loss: 0.0659 - val_loss: 0.0990
Model saved as model_nvidia.h5


# Using generators

In [9]:
samples = []
with open("../data-car-behavioral/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [17]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: #loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset: offset + batch_size]
            
            car_images = []
            steering_angles = []
            for batch_sample in batch_samples:
                steering_center = float(batch_sample[3])
                
                # create adjusted steering angles for the 2 side camera images (left and right one)
                correction = steering_center * 0.1 #10%
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                
                # read in images from center, left and right cameras
                for i in range(3):
                    source_path = batch_sample[i]
                    filename = source_path.split('/')[-1]
                    current_path = '../data-car-behavioral/IMG/' + filename
                    image = cv2.imread(current_path)                    
                    car_images.append(image)
                    
                steering_angles.extend([steering_center, steering_left, steering_right])
            
            X_train = np.array(car_images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

## Training

In [12]:
BATCH_SIZE = 128
train_generator = generator(train_samples, batch_size=BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size=BATCH_SIZE)

row, col, ch = 160, 320, 3

In [21]:
def Nvidia_CNN_gen(train_generator, samples_per_epoch, validation_generator, 
                   num_val_samples, input_shape=(160, 320, 3)):
    model = Sequential()
    # Preprocess incoming data, centered around zero with small standard deviation 
    model.add(Lambda(lambda x: x/127.5 - 1.,
             input_shape=input_shape))
    # crop images 
#     model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    
    # adding 5 convolutions 
    model.add(Conv2D(24, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(36, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid', kernel_initializer='he_normal', activation='relu'))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1164, kernel_initializer='he_normal'))
    model.add(Dense(100, kernel_initializer='he_normal'))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Dense(1, kernel_initializer='he_normal'))

    model.compile(loss='mse', optimizer='adam')
    model.fit_generator(train_generator, steps_per_epoch=samples_per_epoch,
                        epochs=5,
                        validation_data=validation_generator, validation_steps=num_val_samples)

    model.save('model_nvidia_gen.h5')
    print("Model saved as model_nvidia_gen.h5")

In [28]:
def fibonacci():
    numbers_list = []
    while 1:
        if(len(numbers_list) < 2):
            numbers_list.append(1)
        else:
            numbers_list.append(numbers_list[-1] + numbers_list[-2])
        yield numbers_list # change this line so it yields its list instead of 1

our_generator = fibonacci()
my_output = []

for i in range(10):
    my_output = (next(our_generator))
    
print(my_output)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


In [25]:
Nvidia_CNN_gen(train_generator, len(train_samples), validation_generator, len(validation_samples))